In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sqli-xss-dataset/SQLInjection_XSS_CommandInjection_MixDataset.1.0.0.csv
/kaggle/input/sqli-xss-dataset/SQLInjection_XSS_MixDataset.1.0.0.csv


In [13]:
from IPython.display import clear_output
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers # 神經網路層
from tensorflow.keras.layers import Add
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from tensorflow import keras
from IPython.display import SVG,Image
import matplotlib.pyplot as plt
import re

In [14]:
df = pd.read_csv('/kaggle/input/sqli-xss-dataset/SQLInjection_XSS_MixDataset.1.0.0.csv')

In [15]:
def remove_comment(text):
    text = re.sub('//.*?\n|/\*.*?\*/', '', text, flags=re.S)
    text = text.split('--')[0]+"--"
    if '\'' in text :
      removeTarget = text.split('\'')[0]
      text = text.replace(removeTarget, "")
    print("After process: ",text)
    return text

In [16]:
def data2char_index(X, max_len, is_remove_comment = False):
    alphabet = " abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
    result = [] 
    for data in X:
        mat = []
        if is_remove_comment == True : 
          data = remove_comment(data) #移除註解(實驗性)
        for ch in data:
            ch = ch.lower()
            if ch not in alphabet:
                continue
            mat.append(alphabet.index(ch))
        result.append(mat)
    X_char = tf.keras.preprocessing.sequence.pad_sequences(np.array(result, dtype=object), padding='post',truncating='post', maxlen=max_len)
    return X_char

In [17]:
def data_to_symbol_tag(X, max_len ,is_remove_comment = False):
    symbol = " -,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
    result = [] 
    for data in X:
        mat = []
        if is_remove_comment == True : 
          data = remove_comment(data) #移除註解(實驗性)
        for ch in data:
            ch = ch.lower()
            if ch not in symbol:
                mat.append(0)
            else :
                mat.append(symbol.index(ch))
        result.append(mat)
    X_char = tf.keras.preprocessing.sequence.pad_sequences(np.array(result, dtype=object), padding='post',truncating='post', maxlen=max_len)
    return X_char

In [18]:
data = df['Sentence'].values
SQLInjection_label = df['SQLInjection'].values
XSS = df['XSS'].values
Normal = df['Normal'].values
label = np.array([SQLInjection_label,XSS,Normal])
label = label.T

In [19]:
trainX, testX, y_train, y_test = train_test_split(data, label, test_size=0.2)
trainX, x_val, y_train, y_val = train_test_split(trainX, y_train, test_size=0.2)

trainX_text = data2char_index(trainX,max_len=1000)
testX_text = data2char_index(testX,max_len=1000)
trainX_symbol = data_to_symbol_tag(trainX,max_len=1000)
testX_symbol = data_to_symbol_tag(testX,max_len=1000)
x_val_text = data2char_index(x_val,max_len=1000)
x_val_symbol = data_to_symbol_tag(x_val,max_len=1000)

print(trainX_text.shape)
print(testX_text.shape)

(100246, 1000)
(31328, 1000)


In [20]:
def model_struct(max_len):

  pool_siz = 10
  num_heads = 3
  embed_dim = 100


  # 文字輸入層
  input_text = tf.keras.layers.Input(shape=(max_len,))
  # 嵌入層
  embed1 = tf.keras.layers.Embedding(input_dim=70,  output_dim=105, input_length=max_len, trainable=False)(input_text)
  cnn1 = tf.keras.layers.Conv1D(32, 3, padding='same', strides=1, activation='relu')(embed1)
  cnn1 = tf.keras.layers.MaxPooling1D(pool_size=pool_siz)(cnn1)
  # GRU
  GRU0 = layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True, go_backwards=True))(cnn1)
  # 注意力機制
#   MHA0 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=1)(GRU0,GRU0)

  # 符號輸入層
  input_symbol = tf.keras.layers.Input(shape=(max_len,))
  embed2 = tf.keras.layers.Embedding(input_dim=34,  output_dim=51, input_length=max_len, trainable=False)(input_symbol)
  cnn1s = tf.keras.layers.Conv1D(32, 3, padding='same', strides=1, activation='relu')(embed2)
  cnn1s = tf.keras.layers.MaxPooling1D(pool_size=pool_siz)(cnn1s)
  # GRU
  GRU0s = layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True, go_backwards=True))(cnn1s)
  # 注意力機制
#   MHA0s = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=1)(GRU0s,GRU0s)


  # Attension Mix
  CrossAT1 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=1)(GRU0,GRU0s)

  # Connect the outputs of `x` and `y`
  combined = tf.keras.layers.add([GRU0 + CrossAT1, GRU0s  + CrossAT1])

  flat = tf.keras.layers.Flatten()(combined)
  dnn1 = tf.keras.layers.Dense(3, activation="softmax")(flat)

  # 輸出模型
  model = tf.keras.Model(inputs=[input_text, input_symbol], outputs=dnn1)
  return model

In [21]:
model = model_struct(max_len=1000)
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 1000, 105)    7350        ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, 1000, 51)     1734        ['input_4[0][0]']                
                                                                                            

In [22]:
history_log= model.fit([trainX_text,trainX_symbol], y_train, batch_size=64, epochs=10 ,validation_data=([x_val_text, x_val_symbol], y_val))

Epoch 1/10
1567/1567 [==============================] - 406s 254ms/step - loss: 0.1007 - accuracy: 0.9632 - val_loss: 0.0235 - val_accuracy: 0.9932
Epoch 2/10
1567/1567 [==============================] - 394s 252ms/step - loss: 0.0215 - accuracy: 0.9934 - val_loss: 0.0195 - val_accuracy: 0.9945
Epoch 3/10
1567/1567 [==============================] - 409s 261ms/step - loss: 0.0174 - accuracy: 0.9948 - val_loss: 0.0264 - val_accuracy: 0.9931
Epoch 4/10
1567/1567 [==============================] - 412s 263ms/step - loss: 0.0132 - accuracy: 0.9961 - val_loss: 0.0226 - val_accuracy: 0.9943
Epoch 5/10
1567/1567 [==============================] - 396s 253ms/step - loss: 0.0111 - accuracy: 0.9968 - val_loss: 0.0146 - val_accuracy: 0.9967
Epoch 6/10
1567/1567 [==============================] - 395s 252ms/step - loss: 0.0100 - accuracy: 0.9971 - val_loss: 0.0141 - val_accuracy: 0.9961
Epoch 7/10
1567/1567 [==============================] - 394s 252ms/step - loss: 0.0080 - accuracy: 0.9975 - val_

In [23]:
pred = model.predict([testX_text,testX_symbol])
y_pred = np.int64(pred>0.5)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,average='micro')
recall = recall_score(y_test, y_pred,average='micro')
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

979/979 [==============================] - 42s 41ms/step
 Accuracy : 0.9939670582226762 
 Precision : 0.9940622505985635 
 Recall : 0.9939670582226762


In [24]:
# test_sentence=input("Payload:")
# string_index = data2char_index([test_sentence],1000 )
# string_symbol_index = data_to_symbol_tag([test_sentence], 1000 )
# model.predict([string_index ,string_symbol_index])

In [25]:
# Cell 14: Save Model
# Save the trained model
model.save('/kaggle/working/sqli_xss_detection_model.h5')
print("Model saved successfully as 'sqli_xss_detection_model.h5'")

# Optionally, save in SavedModel format (recommended for TensorFlow 2.x)
model.save('/kaggle/working/sqli_xss_detection_model')
print("Model saved successfully in SavedModel format as 'sqli_xss_detection_model'")

# Save model weights only
model.save_weights('/kaggle/working/sqli_xss_model_weights.h5')
print("Model weights saved successfully as 'sqli_xss_model_weights.h5'")

Model saved successfully as 'sqli_xss_detection_model.h5'
Model saved successfully in SavedModel format as 'sqli_xss_detection_model'
Model weights saved successfully as 'sqli_xss_model_weights.h5'
